In [ ]:
from qiskit import QuantumCircuit
import network as nt
from qiskit.circuit import Parameter
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options


# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

In [ ]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils import algorithm_globals
from qiskit.algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler

In [ ]:
from qiskit_optimization.applications.vertex_cover import VertexCover
import networkx as nx

seed = 42
algorithm_globals.random_seed = seed

In [ ]:
graph = nx.random_regular_graph(d=15, n=90, seed=seed)
pos = nx.spring_layout(graph, seed=seed)

In [ ]:
prob = VertexCover(graph)
prob.draw(pos=pos)

In [ ]:
nx.draw(graph, with_labels=True, alpha=0.8, node_size=500)

In [ ]:
adjacency = nx.adjacency_matrix(graph).todense()

N_QUBITS = 3

beta = Parameter("$\\beta$")
qc_mix = QuantumCircuit(N_QUBITS)
for i in range(N_QUBITS):
    qc_mix.rx(2 * beta, i)

qc_mix.draw()

In [ ]:
gamma = Parameter("$\\gamma$")
qc_p = QuantumCircuit(N_QUBITS)
for pair in list(graph.edges()):  # pairs of nodes
    qc_p.rzz(2 * gamma, pair[0], pair[1])
    qc_p.barrier()

qc_p.decompose().draw()

In [ ]:
qc_0 = QuantumCircuit(N_QUBITS)
for i in range(N_QUBITS):
    qc_0.h(i)

qc_0.draw()

In [ ]:
import numpy as np
from qiskit import BasicAer
from qiskit.Air import QuantumInstance
from qiskit.Air.algorithms import QAOA
from qiskit.Air.components.optimizers import COBYLA
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.optimization.problems import QuadraticProgram


n = 4
distances = np.array([[0, 1, 2, 3],
                      [1, 0, 4, 5],
                      [2, 4, 0, 6],
                      [3, 5, 6, 0]])

qubo = QuadraticProgram()
for i in range(n):
    for j in range(n):
        if i != j:
            qubo.binary_var(f'x_{i}_{j}')

for i in range(n):
    qubo.linear_constraint(linear=[1] * n, sense='E', rhs=1, name=f'constraint_{i}')

for i in range(n):
    for j in range(n):
        if i != j:
            qubo.quadratic_constraint(linear=[1, 1], quadratic={f'x_{i}_{j}': -2}, sense='L', rhs=0)


qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qubo)
optimizer = COBYLA(maxiter=100)
qaoa = QAOA(quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator')), optimizer=optimizer, p=2)

quantum_optimizer = MinimumEigenOptimizer(qaoa)

result = quantum_optimizer.solve(qubo)

solution = result.x

print("Optimum solution found:")
print(solution)